# Journal Entries Analysis
A mini data analysis project to examine and analyze the contents of journal entries I have written from 2019 to 2025.

In [7]:
import json
import psycopg2
import glob
from datetime import datetime
import pandas as pd
import numpy as np
import os

In [4]:
DB_PARAMS = {
    "dbname": "journal_db", 
    "user": "postgres",
    "password": "password",
    "host": "localhost"
}

## Inserting data into Postgres DB
Schema was created beforehand on psql

In [ ]:
# Connect to postgres db
conn = psycopg2.connect(**DB_PARAMS)
cursor = conn.cursor()

In [ ]:
# Load JSON files into DB
for file in glob.glob("data/*.json"):  
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)

        # Convert milliseconds timestamp to PostgreSQL TIMESTAMP format
        date_journal = datetime.utcfromtimestamp(data["date_journal"] / 1000)

        # Convert to GMT+8
        date_journal = date_journal.replace(hour=date_journal.hour + 8)

        # Insert into PostgreSQL
        cursor.execute("""
            INSERT INTO journal_entries (id, date_journal, text, tags, mood, sentiment, address, latitude, longitude, weather_description, weather_temp, weather_place)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            data["id"], 
            date_journal, 
            data["text"], 
            data["tags"], 
            data["mood"], 
            data["sentiment"], 
            data["address"], 
            data["lat"], 
            data["lon"], 
            data["weather"]["description"], 
            data["weather"]["degree_c"], 
            data["weather"]["place"]
        ))

In [7]:
# Commit and close connection
conn.commit()
cursor.close()
conn.close()

---

## Initial Cleaning and Exploratory Data Analysis

In [8]:
sql_query = """
SELECT * 
FROM journal_entries;
""" 

In [9]:
# Load the data into a dataframe
conn = psycopg2.connect(**DB_PARAMS)
df = pd.read_sql_query(sql_query, conn)
conn.close()

C:\Users\jmcvi\AppData\Local\Temp\ipykernel_13780\4074806114.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)


## Cleaning
My journaling platform had the option to include several other fields of information with each entry (location, sentiment, etc). 

Since a lot of that data is incomplete or missing, we'll need to clean up the respective columns.

### Entry text
Since the journaling platform processes each text entry as an HTML file, tags are also included in the output. These are removable with a simple RegEx replace.

In [17]:
# Remove HTML tags from the text
df["text"] = df["text"].str.replace(r"<[^>]*>", "", regex=True)

### Mood and Sentiment Columns
Sentiment was an available parameter in my journaling platform that allowed me to specify my mood for that day, ranging from 1-5 on a scale. However, I ended up not specifying my mood for a majority of my entries (was too lazy to do the additional clicks). 

Let's delete these columns for now and extract sentiment later on.

In [19]:
# Drop mood and sentiment columns
df = df.drop(columns=["mood", "sentiment"])

### Latitude and Longitude Location
Replaced invalid values with my home coordinates

### Filling in values
My journaling platform has the option to include your current location, including the current weather if you do choose to include location. However, I ended up not including my location for the majority of my latter entries (was too lazy to click the additional button). 

I now want to fill in all values for weather description and temp with the description and temperature at my home on that given day.
